In [1]:
%run ../../connection.python.py

In [11]:
path_golden_adform = "/user/emma.cambieri/Digitalizzazione/Apr2019/golden_adform_nov_apr.parquet"  ## di EMMA!!
path_model_adform = "/user/emma.cambieri/Digitalizzazione/Apr2019/modelli/model_adform" ## di EMMA!!
path_prediction_adform = "/user/emma.cambieri/Digitalizzazione/Apr2019/prediction/prediction_adform_nov_apr.parquet"  ## di EMMA!!
path_decile_adform_e_variabili = "/user/emma.cambieri/Digitalizzazione/Apr2019/decile_adform_e_variabili.parquet"  ## di EMMA!!

In [6]:
golden_train = spark.read.format('parquet').load(path_golden_adform).persist()

In [7]:
golden_train.count()

2790876

In [11]:
#golden_train.printSchema()

In [12]:
golden_train = golden_train.drop('numero_week', 'impression_tot', 'impression_week', 'impression_weekend',
                                 'impression_mattina', 'impression_pomeriggio','impression_sera',  'impression_notte',
                                 'avg_impression_week','numero_social_provenienza') 

In [15]:
#golden_train.show(10)

In [ ]:
#golden_train = golden_train.fillna(0)

In [16]:
print(len(golden_train.columns))

26


In [18]:
#string_columns= ["social_network", "tipo_device", "ricorrenza_corporate", "ricorrenza_tg24", "ricorrenza_sport",
                      #"ricorrenza_appgtv", "ricorrenza_appxfactor", "ricorrenza_appsport", "ricorrenza_appwsc",
                      #"ricorrenza_corporate_appwsc_appgtv", "ricorrenza_editoriale", "ricorrenza_web", "ricorrenza_app",
                      #"ricorrenza_totale"]
#for c in string_columns:
#    golden_train = golden_train.withColumn(c, golden_train[c].cast('string'))

avg_columns= ["avg_impression_mese","avg_impression_giorno", "perc_impression_mobile",
                    "perc_impression_desktop","perc_impression_tablet","perc_impression_tv_unknown","perc_mobile","perc_desktop",
                    "perc_tablet", "perc_tv_unknown"]

for cols in avg_columns:
    golden_train= golden_train.withColumn(cols, golden_train[cols].cast('float'))
        
integ_columns = golden_train.drop("ext_id_impression","avg_impression_mese","avg_impression_giorno", "perc_impression_mobile",
                    "perc_impression_desktop","perc_impression_tablet","perc_impression_tv_unknown","perc_mobile","perc_desktop",
                    "perc_tablet","perc_tv_unknown","flag_social", "score_digital").columns


for col_int in integ_columns:
  golden_train = golden_train.withColumn(col_int,golden_train[col_int].cast("integer"))

#golden_train.printSchema()

In [19]:
prova= golden_train.filter('score_digital == 0 or score_digital == 1')
print(str("train dataset 0 and 1:"), prova.count()) #dovremo avere intorno ai 200K

prova0= golden_train.filter('score_digital == 0')
print(str("train dataset 0:"), prova0.count()) #intorno ai 80 K

prova1= golden_train.filter('score_digital == 1')
print(str("train dataset 1:"), prova1.count()) #intorno ai 120 K

train dataset 0 and 1: 200011
train dataset 0: 84661
train dataset 1: 115350


In [20]:
testData= golden_train.filter('score_digital == 1 or score_digital is null')

In [21]:
testData.count() #2.407.826

2706215

In [22]:
testData = testData.drop('numero_week', 'impression_tot', 'impression_week', 'impression_weekend',
                                 'impression_mattina', 'impression_pomeriggio','impression_sera',  'impression_notte',
                                 'avg_impression_week', 'numero_social_provenienza', 'score_digital')

In [23]:
print(len(testData.columns))

25


In [24]:
col_test = testData.columns
col_golden_train = golden_train.columns

print(len(set(col_test).intersection(col_golden_train)))

25


In [25]:
#string_columns= ["social_network", "tipo_device", "ricorrenza_corporate", "ricorrenza_tg24", "ricorrenza_sport",
                      #"ricorrenza_appgtv", "ricorrenza_appxfactor", "ricorrenza_appsport", "ricorrenza_appwsc",
                      #"ricorrenza_corporate_appwsc_appgtv", "ricorrenza_editoriale", "ricorrenza_web", "ricorrenza_app",
                      #"ricorrenza_totale"]
#for c in string_columns:
#    golden_train = golden_train.withColumn(c, golden_train[c].cast('string'))

avg_columns= ["avg_impression_mese","avg_impression_giorno", "perc_impression_mobile",
                    "perc_impression_desktop","perc_impression_tablet","perc_impression_tv_unknown", "perc_mobile","perc_desktop",
                    "perc_tablet", "perc_tv_unknown"]

for cols in avg_columns:
    testData= testData.withColumn(cols, testData[cols].cast('float'))
        
integ_columns = testData.drop("ext_id_impression","avg_impression_mese","avg_impression_giorno", "perc_impression_mobile",
                    "perc_impression_desktop","perc_impression_tablet","perc_impression_tv_unknown","perc_mobile","perc_desktop",
                    "perc_tablet","flag_social", "perc_tv_unknown").columns


for col_int in integ_columns:
  testData = testData.withColumn(col_int,testData[col_int].cast("integer"))

#testData.printSchema()

In [26]:
#Import useful packages:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder

### Train, validation and test dataset

In [133]:
#0 da escludere dal training:
golden_train_0_esclusi = golden_train.filter('score_digital == 0')

In [29]:
#golden_train_0_esclusi.count() 

In [30]:
#nuovo golden_train, senza gli 0:
golden_train_1_e_null =golden_train.filter('score_digital == 1 or score_digital is null')

In [59]:
#golden_train_1_e_null.count()

In [33]:
golden_train1 = golden_train.filter('score_digital==1')

In [99]:
#golden_train1.count()

In [36]:
golden_train_null = golden_train.filter('score_digital is null')

In [38]:
#golden_train_null.count()

In [52]:
#proportion of data to be sampled from golen_train_null:
limite_1 = (0.7*golden_train1.count())/0.3
limite = int(limite_1)

In [53]:
golden_train0_70 = golden_train_null.limit(limite)

In [54]:
#golden_train0_70.count()

269150

In [55]:
golden_train1.createOrReplaceTempView('golden_train1')
golden_train0_70.createOrReplaceTempView('golden_train0_70')

In [56]:
# 80% zeri e 20% uni
golden_train0e1 = spark.sql('select * from golden_train1 union all select * from golden_train0_70 ')

In [57]:
#golden_train0e1.count()

384500

In [62]:
golden_train0e1 = golden_train0e1.fillna(0)

In [64]:
#golden_train0e1.printSchema()

In [65]:
steps =[]
CategoricalCol_in = ["flag_social"]
CategoricalCol_out = [ "flag_social_class_vec"]
 
indexers = [StringIndexer(inputCol=x, outputCol=x+'Index')
            for x in CategoricalCol_in ]
 
encoders = [OneHotEncoder(dropLast=False, inputCol=x+"Index", outputCol=y)
for x,y in zip(CategoricalCol_in, CategoricalCol_out)]
steps = [[i,j] for i,j in zip(indexers, encoders)]
steps = [i for sublist in steps for i in sublist]

# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol="score_digital", outputCol="label")
#stages += [label_stringIdx]

numericCols = golden_train0e1.drop("ext_id_impression", "score_digital","flag_social").columns

assemblerInputs = [c + "_class_vec" for c in CategoricalCol_in] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
steps += [assembler, label_stringIdx]

#Pipeline:  
pipeline = Pipeline().setStages(steps)

In [66]:
# Applico pipeline a golden_train:
pipelineModel = pipeline.fit(golden_train0e1)
train = pipelineModel.transform(golden_train0e1)

In [67]:
(training, validation) = train.randomSplit([0.8, 0.2], seed=1)

In [68]:
#training_0 = training.filter('score_digital == 0')
#righe_training_0 = training_0.count()

#training_1 = training.filter('score_digital == 1')
#print (training_1.count())

#print ("percentuale 0 training:", righe_training_0/training.count())
#print ("percentuale 1 training:", training_1.count()/training.count())

#validation_0 = validation.filter('score_digital == 0')
#righe_validation_0 = validation_0.count()

#validation_1 = validation.filter('score_digital == 1')
#print (validation_1.count())

#print ("percentuale 0 validation:", righe_validation_0/validation.count())
#print ("percentuale 1 validation:", validation_1.count()/validation.count())


percentuale 0 training: 0.7004407147639445


## Classification:

In [69]:
#Creo il modello:
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=50, maxDepth = 4)

In [ ]:
#Ottengo la spiegazione di tutti i parametri:
#print(rf.explainParams())

In [70]:
# Run model:
model = rf.fit(training)
predictions_class = model.transform(validation)

In [76]:
model.save(path_model_adform)

In [118]:
# Print degli alberi:
#print(model.toDebugString)

In [120]:
# Select example rows to display.
#predictions_class.select("label", 'prediction').show(10, False)

In [78]:
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions_class)

0.9999618147384152

In [80]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_class)
print("Test Error = %g" % (1.0 - accuracy))

#rfModel = model.stages[2]
#print(rfModel)  # summary only

Test Error = 0.00436096


In [81]:
importance_class= model.featureImportances
print(importance_class)

(25,[0,1,2,3,5,6,7,8,9,10,11,12,13,14,16,17,18,19,20,21,23,24],[0.113730815976,0.184929389309,1.11017329583e-05,0.033126425201,0.0266966103501,0.00564951849834,0.0034122422667,0.0650811088448,0.0587427232203,3.14931545783e-06,0.000788559196609,0.0033673091705,0.00185786988904,0.0914038841684,0.0116869477827,0.012148730905,0.00440662278796,0.00634281494617,0.00313100223002,0.00553715380119,0.353877951657,0.0140680687513])


In [82]:
import pandas as pd
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

In [83]:
#CLASSIFICATION. FEATURE IMPORTANCE
ExtractFeatureImp(importance_class, training, "features").head(50)

,idx,name,score
23,23,numero_impression_social,0.353878
1,1,flag_social_class_vec_1.0,0.184929
0,0,flag_social_class_vec_0.0,0.113731
14,14,numero_device,0.091404
8,8,avg_impression_mese,0.065081
9,9,avg_impression_giorno,0.058743
3,3,numero_giorni,0.033126
5,5,impression_mobile,0.026697
24,24,numero_cookies,0.014068
17,17,numero_desktop,0.012149


### Spreading on customer base

In [121]:
steps1 =[]
CategoricalCol_in = ["flag_social"]
CategoricalCol_out = [ "flag_social_class_vec"]
 
indexers = [StringIndexer(inputCol=x, outputCol=x+'Index')
            for x in CategoricalCol_in ]
 
encoders = [OneHotEncoder(dropLast=False, inputCol=x+"Index", outputCol=y)
for x,y in zip(CategoricalCol_in, CategoricalCol_out)]
steps1 = [[i,j] for i,j in zip(indexers, encoders)]
steps1 = [i for sublist in steps1 for i in sublist]

numericCols = testData.drop("ext_id_impression","flag_social").columns

assemblerInputs = [c + "_class_vec" for c in CategoricalCol_in] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
steps1 += [assembler]

#Pipeline:  
pipeline1 = Pipeline().setStages(steps1)

In [ ]:
#testData = testData.fillna(0)

In [122]:
# Applico pipeline a test:
pipelineModel_final = pipeline1.fit(testData)
final= pipelineModel_final.transform(testData)

In [123]:
predictions_class_test = model.transform(final)

In [124]:
from pyspark.ml.feature import VectorSlicer

slicer = VectorSlicer(inputCol="probability", outputCol="prob_1", indices=[1])

predictions_class_test = slicer.transform(predictions_class_test)

predictions_class_test.select("probability", "prob_1").show()

+--------------------+--------------------+
|         probability|              prob_1|
+--------------------+--------------------+
|[0.99344219826037...|[0.00655780173962...|
|[0.99351718084905...|[0.00648281915094...|
|[0.99833985865467...|[0.00166014134532...|
|[0.99833985865467...|[0.00166014134532...|
|[0.99587805919711...|[0.00412194080288...|
|[0.98616681722560...|[0.01383318277439...|
|[0.99351718084905...|[0.00648281915094...|
|[0.99833985865467...|[0.00166014134532...|
|[0.99344219826037...|[0.00655780173962...|
|[0.99351718084905...|[0.00648281915094...|
|[0.99344219826037...|[0.00655780173962...|
|[0.76383734968001...| [0.236162650319982]|
|[0.99345252180576...|[0.00654747819423...|
|[0.98848309689694...|[0.01151690310305...|
|[0.98421301072403...|[0.01578698927596...|
|[0.99014737613017...|[0.00985262386982...|
|[0.98495668447783...|[0.01504331552216...|
|[0.99351718084905...|[0.00648281915094...|
|[0.98569588216316...|[0.01430411783683...|
|[0.98690501563735...|[0.0130949

In [89]:
#predictions_class_test.printSchema()

In [125]:
predictions_class_test.createOrReplaceTempView('class')

In [126]:
probab_class = spark.sql('SELECT prob_1, COUNT(features) as count_ext_id_impression FROM class GROUP BY prob_1 ORDER BY count_ext_id_impression DESC')

In [127]:
# CLASSIFICATION: DISTRIBUTION OF PROBABILITY
#probab_class.show(1000, False)

+-----------------------+-----------------------+
|prob_1                 |count_ext_id_impression|
+-----------------------+-----------------------+
|[0.006557801739629491] |246341                 |
|[0.0016601413453203222]|224913                 |
|[0.0030662872114683255]|101676                 |
|[0.00235570488849465]  |93651                  |
|[0.009935343533227264] |88737                  |
|[0.0013844140331188458]|75782                  |
|[0.006482819150944841] |71836                  |
|[0.01359935922372438]  |56531                  |
|[0.005423711796313224] |44379                  |
|[0.0055833913714696945]|39714                  |
|[0.011516903103055552] |38879                  |
|[0.00550923073993488]  |38717                  |
|[0.007458083193664094] |38419                  |
|[0.014205429706137852] |30434                  |
|[0.008139361309457778] |30403                  |
|[0.010304537093533047] |27961                  |
|[0.010752905323855429] |26532                  |


In [128]:
probab_class = spark.sql('SELECT prob_1, COUNT(features) as count_ext_id_impression FROM class GROUP BY prob_1 ORDER BY prob_1 DESC')

In [95]:
#probab_class.show(1000, False)

+--------------------+-----------------------+
|prob_1              |count_ext_id_impression|
+--------------------+-----------------------+
|[0.9938624335855163]|4556                   |
|[0.9937933791470427]|53                     |
|[0.9937416963783773]|322                    |
|[0.9936726419399037]|1                      |
|[0.9933703930645322]|92                     |
|[0.9933013386260586]|3                      |
|[0.9932496558573932]|16                     |
|[0.9924014034415858]|276                    |
|[0.9923323490031122]|2                      |
|[0.9922806662344468]|81                     |
|[0.9921351931026041]|8                      |
|[0.9919093629206017]|11                     |
|[0.991876005485372] |52                     |
|[0.9918278369065547]|6                      |
|[0.991755268278233] |5                      |
|[0.9917070996994157]|4                      |
|[0.9913357963855707]|1                      |
|[0.9912150591784317]|2                      |
|[0.990647977

In [129]:
probab_class_prediction = spark.sql('SELECT prediction, COUNT(features) as count_ext_id_impression FROM class GROUP BY prediction ORDER BY count_ext_id_impression DESC')

In [130]:
probab_class_prediction.show(1000, False)

+----------+-----------------------+
|prediction|count_ext_id_impression|
+----------+-----------------------+
|0.0       |2579365                |
|1.0       |126850                 |
+----------+-----------------------+



### Creation of quantiles for column prob_1:

In [131]:
predictions_class_test.select("ext_id_impression", "prob_1").show()

+--------------------+--------------------+
|   ext_id_impression|              prob_1|
+--------------------+--------------------+
|0005f3f18bda8a53a...|[0.00655780173962...|
|0006530d860a31ca2...|[0.00648281915094...|
|0006b83f9d38ac301...|[0.00166014134532...|
|000ae1a773010077d...|[0.00166014134532...|
|000aec8698c204eca...|[0.00412194080288...|
|000fd3e0c929462b3...|[0.01383318277439...|
|001a2224a6e43beef...|[0.00648281915094...|
|001a420df0de04a04...|[0.00166014134532...|
|001da7d5e04f2482f...|[0.00655780173962...|
|001e2193b90bffb5b...|[0.00648281915094...|
|0028e80ef65484c56...|[0.00655780173962...|
|002bd395a3f7ca4ec...| [0.236162650319982]|
|002e280652ccfb8b0...|[0.00654747819423...|
|0034732e24e76cc3a...|[0.01151690310305...|
|00376aa9ec875e1e5...|[0.01578698927596...|
|003bd909360cc50de...|[0.00985262386982...|
|003f67404607a7474...|[0.01504331552216...|
|0040ef69407f61914...|[0.00648281915094...|
|00426deec96b82d73...|[0.01430411783683...|
|00445b35f036de3ae...|[0.0130949

In [134]:
#0 da escludere dal training:
#golden_train_0_esclusi = golden_train.filter('score_digital == 0')
golden_train_0_esclusi.select("ext_id_impression","score_digital").show()

+--------------------+-------------+
|   ext_id_impression|score_digital|
+--------------------+-------------+
|00c25fa3cdfeda4af...|          0.0|
|011b58b13ce8d79b6...|          0.0|
|01311ff44add3945b...|          0.0|
|0239ea93c8ba9db0e...|          0.0|
|0261f9a4b52eee5fa...|          0.0|
|030eb141ee6916a27...|          0.0|
|04a609e713f788414...|          0.0|
|04b14798f630d65f4...|          0.0|
|0581ad2e5ce5f5a8f...|          0.0|
|05a3d479558d8ad32...|          0.0|
|06875c0facb5006a5...|          0.0|
|07155432bf41e5807...|          0.0|
|07e93ebb8fb1f6b80...|          0.0|
|088c376e730314643...|          0.0|
|0a5b02590d83a11ed...|          0.0|
|0aad0dc51f2b55c8c...|          0.0|
|0b05bb6b399ecba5e...|          0.0|
|0b5bd059c5fe5e43e...|          0.0|
|0decd80710c7530d8...|          0.0|
|0e3cadfda8d9643cc...|          0.0|
+--------------------+-------------+
only showing top 20 rows



In [135]:
golden_train_0_esclusi= golden_train_0_esclusi.withColumnRenamed("score_digital","prob_1_int")
golden_train_0_esclusi = golden_train_0_esclusi.withColumn('prob_1_int',golden_train_0_esclusi['prob_1_int'].cast("long"))

In [136]:
#golden_train_0_esclusi.printSchema()

In [137]:
probab_col = ['prob_1_int']
assembler_golden0 = VectorAssembler(inputCols=probab_col, outputCol="prob_1")

In [138]:
zeri_esclusi = assembler_golden0.transform(golden_train_0_esclusi)

In [139]:
zeri_esclusi = zeri_esclusi.drop('prob_1_int')

In [140]:
#zeri_esclusi.printSchema()

In [141]:
#predictions_class_test.printSchema()

In [142]:
predictions_class_test1=predictions_class_test.drop('features', 'rawPrediction', 'probability', 'prediction', 'flag_socialIndex',
                                                   'flag_social_class_vec')

In [143]:
#predictions_class_test1.printSchema()

In [ ]:
#from pyspark.sql.functions import udf, col
#from pyspark.sql.types import ArrayType, DoubleType

#def to_array(col):
#    def to_array_(v):
#        return v.toArray().tolist()
#    return udf(to_array_, ArrayType(DoubleType()))(col)

In [ ]:
#predictions_class_test1 = predictions_class_test1.withColumn("prob_1_prova", to_array(col("prob_1")))

In [ ]:
#predictions_class_test1['prob_1', 'prob_1_prova'].show()

In [ ]:
#from pyspark.sql.functions import udf

#vector_udf = udf(lambda vector: float(vector))

#predictions_class_test1 = predictions_class_test1.withColumn('prob_1_int', vector_udf('prob_1'))

In [ ]:
#udf1 = udf(lambda x : float(x[0]),DoubleType())
#predictions_class_test1 = predictions_class_test1.withColumn('prob_1_double', udf1('prob_1'))

### Unione dei dataset

In [144]:
predictions_class_test1.createOrReplaceTempView('predictions_class_test1')
zeri_esclusi.createOrReplaceTempView('zeri_esclusi')

In [145]:
zeri_esclusi.dtypes

[('ext_id_impression', 'string'),
 ('numero_mesi', 'int'),
 ('numero_giorni', 'int'),
 ('impression_tv_unknown', 'int'),
 ('impression_mobile', 'int'),
 ('impression_desktop', 'int'),
 ('impression_tablet', 'int'),
 ('avg_impression_mese', 'float'),
 ('avg_impression_giorno', 'float'),
 ('perc_impression_tv_unknown', 'float'),
 ('perc_impression_mobile', 'float'),
 ('perc_impression_desktop', 'float'),
 ('perc_impression_tablet', 'float'),
 ('numero_device', 'int'),
 ('numero_tv_unknown', 'int'),
 ('numero_mobile', 'int'),
 ('numero_desktop', 'int'),
 ('numero_tablet', 'int'),
 ('perc_mobile', 'float'),
 ('perc_desktop', 'float'),
 ('perc_tablet', 'float'),
 ('perc_tv_unknown', 'float'),
 ('numero_impression_social', 'int'),
 ('flag_social', 'double'),
 ('numero_cookies', 'int'),
 ('prob_1', 'vector')]

In [146]:
predictions_finale = spark.sql('select * from predictions_class_test1 union all select * from zeri_esclusi ')

In [147]:
predictions_finale.count()

2790876

In [149]:
#predictions_finale.groupBy('prob_1').count().sort('count', ascending=False).show()

In [ ]:
#predictions_class_test1.createOrReplaceTempView('predictions_class_test1')
#quantiles = spark.sql("select percentile_approx(prob_1_int, 0.25) as q25, percentile_approx(prob_1_int, 0.50) as mediana, percentile_approx(prob_1_int, 0.75) as q75, percentile_approx(prob_1_int, 0.95) as q95, percentile_approx(prob_1_int, 1) as max from predictions_class_test1")

### Creazione decili

In [150]:
predictions_finale.createOrReplaceTempView('predictions_finale')

In [151]:
probab_class_asc = spark.sql('SELECT ext_id_impression, prob_1 FROM predictions_finale ORDER BY prob_1').persist()

In [152]:
probab_class_asc.count()

2790876

In [153]:
probab_class_asc.createOrReplaceTempView('probab_class_asc')

In [154]:
probab_class_asc_indexed = spark.sql('select row_number() over (order by prob_1) as num, * from probab_class_asc').persist()

In [155]:
#probab_class_asc_indexed.show(100)

+---+--------------------+------+
|num|   ext_id_impression|prob_1|
+---+--------------------+------+
|  1|00c25fa3cdfeda4af...| [0.0]|
|  2|011b58b13ce8d79b6...| [0.0]|
|  3|01311ff44add3945b...| [0.0]|
|  4|0239ea93c8ba9db0e...| [0.0]|
|  5|0261f9a4b52eee5fa...| [0.0]|
|  6|030eb141ee6916a27...| [0.0]|
|  7|04a609e713f788414...| [0.0]|
|  8|04b14798f630d65f4...| [0.0]|
|  9|0581ad2e5ce5f5a8f...| [0.0]|
| 10|05a3d479558d8ad32...| [0.0]|
| 11|06875c0facb5006a5...| [0.0]|
| 12|07155432bf41e5807...| [0.0]|
| 13|07e93ebb8fb1f6b80...| [0.0]|
| 14|088c376e730314643...| [0.0]|
| 15|0a5b02590d83a11ed...| [0.0]|
| 16|0aad0dc51f2b55c8c...| [0.0]|
| 17|0b05bb6b399ecba5e...| [0.0]|
| 18|0b5bd059c5fe5e43e...| [0.0]|
| 19|0decd80710c7530d8...| [0.0]|
| 20|0e3cadfda8d9643cc...| [0.0]|
| 21|0e59f953e3bb2dedf...| [0.0]|
| 22|0fc31d081ec61d66b...| [0.0]|
| 23|10ee37834c585d58d...| [0.0]|
| 24|10f70840c0cdf0f7e...| [0.0]|
| 25|11fa9bde02e952b20...| [0.0]|
| 26|1201e96d9346e007f...| [0.0]|
| 27|124f4b8e0

In [157]:
#probab_class_asc_indexed.printSchema()

In [ ]:
#display(probab_class_asc_indexed.count()/10)
#display(297136*2)
#display(297136*3)
#display(297136*4)
#display(297136*5)
#display(297136*6)
#display(297136*7)
#display(297136*8)
#display(297136*9)
#display(probab_class_asc_indexed.count())

In [170]:
lung = int(probab_class_asc.count()/10)
lung

279087

In [171]:
probab_class_asc_indexed_2 = probab_class_asc_indexed.withColumn('lunghezza', lit(lung))

In [164]:
#probab_class_asc_indexed_2.show(100)
#probab_class_asc_indexed_2.printSchema()

In [172]:
probab_class_asc_indexed_2.createOrReplaceTempView('probab_class_asc_indexed')

In [173]:
probab_class_asc_index_decili = spark.sql('SELECT *, CASE WHEN (num between 0 and lunghezza) THEN 1 WHEN (num between lunghezza and lunghezza*2) THEN 2 WHEN (num between lunghezza*2 and lunghezza*3) THEN 3 WHEN (num between lunghezza*3 and lunghezza*4) THEN 4 WHEN (num between lunghezza*4 and lunghezza*5) THEN 5 WHEN (num between lunghezza*5 and lunghezza*6) THEN 6 WHEN (num between lunghezza*6 and lunghezza*7) THEN 7 WHEN (num between lunghezza*7 and lunghezza*8) THEN 8 WHEN (num between lunghezza*8 and lunghezza*9) THEN 9 WHEN (num > lunghezza*9) THEN 10 ELSE NULL END AS decile FROM probab_class_asc_indexed')

In [181]:
#probab_class_asc_index_decili.show(100, False)

In [179]:
probab_class_asc_index_decili_1= probab_class_asc_index_decili.drop('lunghezza')

In [183]:
probab_class_asc_index_decili_1.write.parquet(path_prediction_adform)

In [175]:
#bin_count = probab_class_asc_index_decili.groupby('decile').count()

In [176]:
#bin_count.show()

+------+------+
|decile| count|
+------+------+
|     1|279087|
|     2|279087|
|     3|279087|
|     4|279087|
|     5|279087|
|     6|279087|
|     7|279087|
|     8|279087|
|     9|279087|
|    10|279093|
+------+------+



In [184]:
#decili_e_probability = probab_class_asc_index_decili_1.filter('num == 297136 or num== 594272 or num == 891408 or num==1188544 or num==1485680 or num == 1782816 or num ==2079952 or num ==2377088 or num == 2674224 or num == 2971364')

In [185]:
#decili_e_probability.show(100, False)

### Unisco decili e variabili di Adform

In [3]:
df_originale = spark.read.format('parquet').load(path_golden_adform).persist()

In [6]:
df_originale.printSchema()

root
 |-- ext_id_impression: string (nullable = true)
 |-- numero_mesi: long (nullable = true)
 |-- numero_week: long (nullable = true)
 |-- numero_giorni: long (nullable = true)
 |-- impression_tot: long (nullable = true)
 |-- impression_tv_unknown: long (nullable = true)
 |-- impression_mobile: long (nullable = true)
 |-- impression_desktop: long (nullable = true)
 |-- impression_tablet: long (nullable = true)
 |-- impression_week: long (nullable = true)
 |-- impression_weekend: long (nullable = true)
 |-- impression_mattina: long (nullable = true)
 |-- impression_pomeriggio: long (nullable = true)
 |-- impression_sera: long (nullable = true)
 |-- impression_notte: long (nullable = true)
 |-- avg_impression_mese: double (nullable = true)
 |-- avg_impression_week: double (nullable = true)
 |-- avg_impression_giorno: double (nullable = true)
 |-- perc_impression_tv_unknown: double (nullable = true)
 |-- perc_impression_mobile: double (nullable = true)
 |-- perc_impression_desktop: doub

In [4]:
df_decile_adform = spark.read.format('parquet').load(path_prediction_adform).persist()

In [5]:
#df_decile_adform.printSchema()

root
 |-- num: integer (nullable = true)
 |-- ext_id_impression: string (nullable = true)
 |-- prob_1: vector (nullable = true)
 |-- decile: integer (nullable = true)



In [7]:
df_originale.createOrReplaceTempView('df_originale')
df_decile_adform.createOrReplaceTempView('df_decile_adform')

In [9]:
df_joined = spark.sql('select distinct df_originale.*, df_decile_adform.decile as decile_adform from df_originale left join df_decile_adform on df_originale.ext_id_impression = df_decile_adform.ext_id_impression')

In [10]:
df_joined = df_joined.drop('score_digital')

In [12]:
df_joined.write.parquet(path_decile_adform_e_variabili)

In [13]:
#a = spark.read.format('parquet').load(path_decile_adform_e_variabili).persist()

In [20]:
#a.printSchema()

In [19]:
#a_count = a.groupby('decile_adform').count()
#a_count.show(15)

+-------------+------+
|decile_adform| count|
+-------------+------+
|            1|279087|
|            6|279087|
|            3|279087|
|            5|279087|
|            9|279087|
|            4|279087|
|            8|279087|
|            7|279087|
|           10|279093|
|            2|279087|
+-------------+------+

